<a href="https://colab.research.google.com/github/David-Marinho/faculdade/blob/main/projeto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalaçao de bibliotecas e ferramentas
pip install tensorflow

In [ ]:
uri = os.getcwd()
rmdir /content/tumores/ipynb_checkpoints

In [2]:
import os
import matplotlib as plt
import tensorflow as tf

In [4]:
dataset_dir = os.path.join(os.getcwd(), 'tumores')
dataset_train_dir = os.path.join(dataset_dir, 'train')
dataset_menigioma_len = len(os.listdir(os.path.join(dataset_train_dir, 'menigioma')))
dataset_validation_dir = os.path.join(dataset_dir, 'validation')

print(f'Existem {dataset_menigioma_len} fotos na pasta de menigioma')

Existem 608 fotos na pasta de menigioma


In [5]:
image_width = 512
image_height = 512
image_color_chanel = 3
image_color_channel_size = 255
image_size = (image_width, image_height)
image_shape = (512, 512, 3)

batch_size = 64
epochs = 20
learning_rate = 0.0001
class_names = ['meningioma', 'glioma', 'tumor hipofisário']

In [6]:
dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_train_dir,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True
)

Found 2764 files belonging to 3 classes.


In [7]:
dataset_validation = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_validation_dir,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True
)

Found 300 files belonging to 3 classes.


In [8]:
dataset_validation_cardinality = tf.data.experimental.cardinality(dataset_validation)
dataset_validation_batches = dataset_validation_cardinality // 5

dataset_test = dataset_validation.take(dataset_validation_batches)

In [9]:
num_classes = len(class_names)

model = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1. / image_color_channel_size, input_shape=image_shape),
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255., input_shape=(image_width, image_height, 1)),

    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 512, 512, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 512, 512, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 64)      1

In [10]:
history = model.fit(
    dataset_train,
    validation_data=dataset_validation,
    epochs=epochs
)

Epoch 1/20
44/44 [==============================] - 1023s 23s/step - loss: 0.8015 - accuracy: 0.6415 - val_loss: 0.4420 - val_accuracy: 0.8500
Epoch 2/20
38/44 [========================>.....] - ETA: 2:18 - loss: 0.4753 - accuracy: 0.8063

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
class_names = [name for name in dataset_train.class_names if name != '.ipynb_checkpoints']
print(class_names)
dataset_train.class_names